In [ ]:
import sys
# Add the badminton package to the path
sys.path.insert(0, '/work/vdark/badminton')

# Define file paths
videos_base = "/work/badminton/VB_DATA/videos/"
poses_base  = "/work/vdark/badminton/badminton/VB_DATA/poses/"

# OpenAI Badminton Analysis

This notebook demonstrates badminton shot analysis using OpenAI's language models.
It processes pose data to generate shot descriptions and classifications.

In [ ]:
#!pip install openai

from openai import OpenAI
import sys
import os
import csv

# Import prompt modules
import badminton.llm_analysis.bd_prompt as bd_prompt
import badminton.llm_analysis.shot_classification_prompt as scp

# Import VideoPoseDataset
from badminton.data.video_pose_dataset import VideoPoseDataset

# Import utility functions
from badminton.utilities.coco_keypoints import create_keypoints_dict

# Import analysis components
from badminton.data.pose_data_loader import PoseDataLoader
from badminton.visualization.pose_visualizer import PoseVisualizer
from badminton.features.pose_feature_extractor import PoseFeatureExtractor
from badminton.analysis.shot_descriptor import ShotDescriptor

## Annotate Video with Pose Data

Load the video and pose data for analysis.
Create an annotated video with pose overlays.

In [ ]:
video_file = videos_base + "05_Drop_Shot/2022-09-01_17-49-36_dataset_set1_058_003682_003721_B_05.mp4"
pose_file = poses_base + "05_Drop_Shot/2022-09-01_17-49-36_dataset_set1_058_003682_003721_B_05.csv"

# Create VideoPoseDataset
vpd = VideoPoseDataset(poses_path=pose_file, video_path=video_file)

print(f"Dataset loaded successfully!")
print(f"Number of frames: {len(vpd)}")
print(f"Video metadata: {vpd.get_dataset_summary()}")

# Create annotated video
vpd.annotate_video_with_poses(
    output_path="annotated_video.mp4", 
    include_bboxes=False, 
    players=['green', 'blue']
)

print("Annotated video created: annotated_video.mp4")

Dataset loaded successfully!
Number of frames: 39
Video metadata: {'poses_path': '/Users/chanakyd/work/vdark/badminton/badminton/VB_DATA/poses/05_Drop_Shot/2022-09-01_17-49-36_dataset_set1_058_003682_003721_B_05.csv', 'video_path': '/Users/chanakyd/work/badminton/VB_DATA/videos/05_Drop_Shot/2022-09-01_17-49-36_dataset_set1_058_003682_003721_B_05.mp4', 'frame_count': 39, 'fps': 30.0, 'duration': 1.3, 'frame_shape': (720, 1280, 3), 'total_frames': 39}


## Poselets Extraction

Poselets provide the orientation of three keypoints relative to each other.

In [4]:
# Extract poselets
poselets_green = vpd.get_poselets_for_player(player='green')
poselets_blue = vpd.get_poselets_for_player(player='blue')

print(f"Extracted {len(poselets_green)} frames of poselets for green player")
print(f"First frame poselets: {poselets_green[0]}")

# Use the feature extractor directly
feature_extractor = PoseFeatureExtractor()
player_data = vpd.data_loader.get_player_data('green')
poselets_direct = feature_extractor.extract_poselets_for_player(player_data)

print(f"\nDirect extraction: {len(poselets_direct)} frames")
print(f"Results match: {poselets_green == poselets_direct}")

Extracted 39 frames of poselets for green player
First frame poselets: {'left_arm': 'P_90_180', 'left_leg': 'P_30_90', 'left_torso': 'P_240_270', 'right_arm': 'P_120_120', 'right_leg': 'P_90_90', 'right_torso': 'P_270_210'}

Direct extraction: 39 frames
Results match: True


## Shot Description Generation

Generate natural language descriptions of the shot.

In [5]:
# Generate shot description
shot_description = vpd.get_shot_description_for_player(player='green')

print("Shot description for green player:")
print(shot_description[:500] + "..." if len(shot_description) > 500 else shot_description)

# Use the shot descriptor directly
shot_descriptor = ShotDescriptor()
poselets = vpd.get_poselets_for_player('green')
description_direct = shot_descriptor.generate_shot_description(poselets, poses_path=pose_file)

print(f"\nDescriptions match: {shot_description == description_direct}")

Shot description for green player:
Position: FrontCourt
Frame,left_arm,left_leg,left_torso,right_arm,right_leg,right_torso
1,P_90_180,P_30_90,P_240_270,P_120_120,P_90_90,P_270_210
2,P_30_270,P_30_90,P_240_240,P_90_150,P_90_90,P_270_210
3,P_0_90,P_30_60,P_240_240,P_120_150,P_90_90,P_270_210
4,P_30_270,P_30_90,P_240_270,P_90_150,P_90_90,P_270_210
5,P_0_90,P_30_90,P_240_270,P_120_150,P_90_90,P_330_210
6,P_0_90,P_30_90,P_240_270,P_120_90,P_90_90,P_270_210
7,P_60_0,P_30_60,P_240_270,P_120_90,P_90_90,P_210_300
8,P_60_0,P_30_60,...

Descriptions match: True


## Prompt Generation

Generate the prompt to send to OpenAI

In [ ]:
# Generate prompt for shot classification
prompt = scp.SC_BASE_PROMPT + scp.SC_INPUT_PROMPT + vpd.get_shot_description_for_player(player='green')


## Run inference on OpenAI

Use OpenAI's language models for shot classification.

In [ ]:
# OpenAI client setup (add your API key)
client = OpenAI(
  api_key="Enter Your Key Here"
)

#model = "gpt-oss-120b"
model="gpt-5-nano"


print("Prompt generated successfully!")
print(f"Prompt length: {len(prompt)} characters")
print("\nFirst 200 characters of prompt:")
print(prompt[:200] + "...")

response = client.responses.create(
    model=model,
    input=prompt,
    store=True)
 
print("")
print("OpenAI Response:")
print(response.output_text)

Prompt generated successfully!
Prompt length: 8509 characters

First 200 characters of prompt:
You are a badminton assistant coach trained to recognize badminton shot types from pose sequences.

Task:

Given: A court position A sequence of pose frames encoded as orientation tokens
Output: Predi...
OpenAI Response:
{"predictions": [
  {
    "label": "05_Drop_Shot",
    "confidence": 0.62,
    "evidence": "In FrontCourt, the right arm remains mid-range (roughly P_120_120 to P_120_150) with only small elevation changes and the right torso shows minimal rotation; this matches a short, controlled drop shot rather than a smash (which would feature a high cocked right arm and pronounced forward torso rotation) or a serve (not applicable in FrontCourt). Confidence moderate due to lack of clear hallmark of extreme motion but overall fit."
  }
]}
